In [1]:
import pandas as pd
import random
import csv
from nltk.corpus import wordnet, stopwords
import time


In [3]:
stop_words = set(stopwords.words('english'))

def sr_method(input_sent, change_percent):
    words = input_sent.split(' ')
    change_size = int(len(words)*change_percent)
    new_words = words.copy()
    idx_non_sw = [i for i in range(len(words)) if words[i] not in stop_words]
    chosen_idx = random.sample(idx_non_sw, change_size)
    for i in chosen_idx:
        chosen_word = words[i]
        synonym = get_synonym(chosen_word)
        if synonym:
            words[i] = synonym

    new_sentence = ' '.join(words)
    

    return new_sentence

def get_syn(word):
    syns = wordnet.synsets(word)
    if syns:
        idx = random.randint(0, len(syns)-1)
        syn = syns[idx]
        lemmas = syn.lemmas()
        l_idx = random.randint(0, len(lemmas)-1)
        syn = lemmas[l_idx]
        syn = syn.name().replace("_", " ").replace("-", " ").lower()
        syn = "".join([char for char in syn if char.isalpha() or char == " "])
    else:
        return word
    return syn

In [4]:
def calculate_stats(beginning, reset, now, i, trans_count, unchanged, num):
    per = (now-beginning)/(i+1)
    mega_time = (per)*(num - i)
    print("Stats every 100 items")
    print("Total translated = {}".format(trans_count))
    print("Total unchanged = {}".format(unchanged))  
    print("Overall time elapsed = {} seconds".format(now - beginning))
    print("Overall time since last calc = {} seconds".format(now-reset))
    print("estimated time left = {} seconds".format(mega_time))
    print()

In [ ]:
from_files = ['LIAR.csv', 'Kaggle2_Mixed_a.csv','Kaggle2_Mixed_b.csv']
to_files = ['LIAR_SR.csv', 'Kaggle2_Mixed_a_SR.csv','Kaggle2_Mixed_b_SR.csv']
cp = .3 #what percent of a given text to change

for i in range(len(from_files)):
    file_path = "../../Augmentation Data/semi_cleaned/{}".format(from_files[i])
    df_input = pd.read_csv(file_path)
    df_input.drop(['Unnamed: 0'], inplace=True,axis=1)
    df_input.head()
    df_output = df_input.copy()
    num = df_input.shape[0]
    unchanged = 0
    sr_count = 0

    beginning = time.time()
    reset = beginning 
    for i in range(num):
        og_text = df_input['clean_text'].iloc[i]
        try:
            sr = sr_method(og_text,cp)
            df_output.loc[i] = [sr,df_input.clean_title[i],df_input.label[i]]
            sr_count += 1
        except:
            unchanged += 1
            pass
        if (i+1) % 500 == 0: #calculate stats every 500 
            calculate_stats(beginning, reset, time.time(), i, sr_count, unchanged, num)
            reset = time.time()

    end = time.time()
    output_path = "../../Augmentation Data/{}".format(to_files[i])

In [2]:
file_path = "../../Augmentation Data/LIAR_SR.csv"
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,clean_text,clean_title,label
0,0,enunciate the annies listing political group s...,says the annies list political group supports ...,0
1,1,when did the decline of char start? it come ou...,when did the decline of coal start? it started...,1
2,2,sir edmund percival hillary dewitt clinton con...,"hillary clinton agrees with john mccain ""by vo...",1
3,3,health care reclaim legislation is likely to m...,health care reform legislation is likely to ma...,0
4,4,the economical turnaround set about at the ter...,the economic turnaround started at the end of ...,1


In [3]:
file_path = "../../Augmentation Data/LIAR_BT.csv"
df2 = pd.read_csv(file_path)
df2.head()

,Unnamed: 0,clean_text,clean_title,label
0,0,The group supports third-trimester abortions o...,says the annies list political group supports ...,0
1,1,When did the decline of coal begin? it began w...,when did the decline of coal start? it started...,1
2,2,"Hillary Clinton agrees with John McCain ""by vo...","hillary clinton agrees with john mccain ""by vo...",1
3,3,The health-care reform law is likely to requir...,health care reform legislation is likely to ma...,0
4,4,The economic turnaround began at the end of my...,the economic turnaround started at the end of ...,1


In [4]:
file_path = "../../Augmentation Data/semi_cleaned/LIAR.csv"
df3 = pd.read_csv(file_path)
df3.head()

,Unnamed: 0,clean_text,clean_title,label
0,0,says the annies list political group supports ...,says the annies list political group supports ...,0
1,1,when did the decline of coal start? it started...,when did the decline of coal start? it started...,1
2,2,"hillary clinton agrees with john mccain ""by vo...","hillary clinton agrees with john mccain ""by vo...",1
3,3,health care reform legislation is likely to ma...,health care reform legislation is likely to ma...,0
4,4,the economic turnaround started at the end of ...,the economic turnaround started at the end of ...,1


In [6]:
for i in range(3):
    print("Original")
    print(df3.clean_text.iloc[i])
    print("SR")
    print(df.clean_text.iloc[i])
    print("BT")
    print(df2.clean_text.iloc[i])
    print()

Original
says the annies list political group supports third-trimester abortions on demand.
SR
enunciate the annies listing political group supports third-trimester miscarriage on demand.
BT
The group supports third-trimester abortions on demand.

Original
when did the decline of coal start? it started when natural gas took off that started to begin in (president george w.) bushs administration.
SR
when did the decline of char start? it come out when instinctive brag subscribe off that come out to commence in (president george ii w.) bushs administration.
BT
When did the decline of coal begin? it began when natural gas started flowing in (President George W.) Bush's administration.

Original
hillary clinton agrees with john mccain "by voting to give george bush the benefit of the doubt on iran."
SR
sir edmund percival hillary dewitt clinton concord with king john mccain "by vote to give george bush the benefit of the doubt on iran."
BT
Hillary Clinton agrees with John McCain "by voting